In [2]:
pip install agentscope

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: loguru
    Found existing installation: loguru 0.7.2
    Uninstalling loguru-0.7.2:
      Successfully uninstalled loguru-0.7.2
  Attempting uninstall: Flask
    Found existing installation: Flask 3.0.3
    Uninstalling Flask-3.0.3:
      Successfully uninstalled Fla

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import agentscope
from agentscope.agents import DialogAgent, UserAgent
from agentscope.message import Msg
import io
import contextlib

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [7]:
# utils

# 正则trim + 截取
def get_code(string):
    string = string.replace('```python',"```").replace('```py',"```").replace('\n\n',"\n")
    # 截取code
    start_index = string.find("```") + 3
    end_index = string.rfind("```")
    response_trim_code = string[start_index:end_index]
    return response_trim_code

def run_py_code(x_code,show_code):    
        code = x_code['content']
        # trim
        print('----------- 正则化 ---------------')
        response_code = get_code(code) # 正则
        if show_code == True:
            print(response_code)
        print('----------- 运行 ---------------')
        # output
        with io.StringIO() as buf, contextlib.redirect_stdout(buf):
            exec(response_code)
            print('----------- 保存输出 ---------------')
            response_code_output = buf.getvalue()
            
        return response_code_output

In [8]:
# key
api_zz_chatglm = "427a3db14abb319a2893d278f947ee3a.tk8jNziddv2uuCpt"
api_zz_qwen = "sk-f838127f25b646109a538b349e1cf7f0"
url_chatglm = "https://open.bigmodel.cn/api/paas/v4/"
url_qwen = "https://dashscope.aliyuncs.com/compatible-mode/v1"
model_chatglm1  = "glm-4"  # ￥0.100000 元/1000 tokens
model_chatglm2  = "GLM-3-Turbo" # 0.001元 / 千tokens
model_qwen  = "qwen-max"   # qwen-plus  qwen-max  qwen-max-0428 qwen-turbo

In [15]:
# 定义 agentscope
model_config = { "model_type": "dashscope_chat",
                "config_name": "qwen",
                "model_name": model_qwen,
                "api_key": api_zz_qwen,
                "generate_args": {"temperature": 0.9 }  }
agentscope.init(model_configs=model_config)  # 读取模型配置


# 定义agent
analyst = DialogAgent(name="数据分析师", 
                          model_config_name="qwen",  # model_config["config_name"]
                          sys_prompt="""
                          # role ：
                          你是1个经验丰富的数据分析师，你的主要职责是根据python数据分析代码的输出结果，输出简洁清晰的数据分析报告
                          你的数据分析思路：①数据处理 ②统计分析 ③对比分析 ④数据可视化
                          
                          # 数据样例（data）
                          import pandas as pd
                          data = {'date': ['20240501', '20240502', '20240503', '20240504'],
                                  'Sales_volume_Shanghai': [100, 100, 200, 400],
                                  'Sales_volume_Beijing': [200, 300, 200, 200],
                                  'Stores_number_Shanghai': [5, 6, 7, 2],
                                  'Stores_number_Beijing': [10, 11, 12, 10]}
                           df = pd.DataFrame(data)
                          
                          # task：
                          1.你需要先向程序员描述你的需求（详细解释每一个数据处理步骤，他需要一次性提供所有步骤的代码），要求他用python来帮助实现
                          2.当程序员把写python代码块交给你，你需要仔细核对代码是否正确，如果代码正确，请直接输出json字段{分析师:已完成需求沟通}(此时你不需要输出其他字符)
                          ；如果不正确，你需要让程序员调整代码，然后把代码重新发给你核对，你再重新核对。

                          """
                           )

coder = DialogAgent(name="程序员", 
                          model_config_name="qwen",  #   model_config_name =  model_config["config_name"]
                          sys_prompt="""
                          #角色（role）：：
                          你是1个经验丰富的程序员，你的工作职责是根据分析师的需求，根据数据样例的格式，提供准确的python示例代码。
                          
                          # 输出要求（Answer）：
                          1.请你考虑可能出现的问题，尽可能给出大概率能成功的代码；
                          2.你需要以代码块的格式输出答案，你输出的代码需要可以直接在python环境运行
                          3.给我发送代码的时候，请永远将所有代码放在一个代码块发给我，不能分好几个，你需要确保每一行都有注释
                          4.如果回答问题质量很好，我会给你10000美元
                          5.每次回答问题之前，都要认真阅读所有要求三遍，确认我的所有要求！
                          6.你每次做回答不能对问题条件做任何假设，请尽可能给出案例回答
                          7.请把每一步骤的代码运行结果都打印出来
                          8.请不要假设存在任何文件，一切代码都需要基于根据数据样例的格式（dataframe）来运行
                          9.请保持用英文输出全部内容
                    
                          
                          # 少样本示例（Case）：
                          import pandas as pd
                          import numpy as np
                          df = pd.DataFrame(np.random.rand(10, 2), columns=['aa', 'cc'])
                          print(df.describe())
                          
                          # 数据样例
                          import pandas as pd
                          data = {'date': ['20240501', '20240502', '20240503', '20240504'],
                                  'Sales_volume_Shanghai': [100, 100, 200, 400],
                                  'Sales_volume_Beijing': [200, 300, 200, 200],
                                  'Stores_number_Shanghai': [5, 6, 7, 2],
                                  'Stores_number_Beijing': [10, 11, 12, 10]}
                           df = pd.DataFrame(data)
                          
                            task：
                           你需要先确认自己已经充分了数据分析师的需求，再提供正确的python代码块，
                           如果分析师要求你调整，你需要耐心的听从他的建议，重新提供完整且正确的python代码块
                           """
                           )

# 创建用户智能体userAgent 
userAgent = UserAgent()
print('agent ready')

2024-05-20 15:27:09.573 | WARNING  | agentscope.models:read_model_configs:163 - config_name [qwen] already exists.
2024-05-20 15:27:09.575 | INFO     | agentscope.models:read_model_configs:170 - Load configs for model wrapper: qwen
2024-05-20 15:27:09.579 | INFO     | agentscope.models.model:__init__:201 - Initialize model by configuration [qwen]
2024-05-20 15:27:09.585 | INFO     | agentscope.models.model:__init__:201 - Initialize model by configuration [qwen]
agent ready


In [16]:
import pandas as pd
data = {'date': ['20240501', '20240502', '20240503', '20240504'],
                                  'Sales_volume_Shanghai': [100, 100, 200, 400],
                                  'Sales_volume_Beijing': [200, 300, 200, 200],
                                  'Stores_number_Shanghai': [5, 6, 7, 2],
                                  'Stores_number_Beijing': [10, 11, 12, 10]}
df = pd.DataFrame(data)


# communicate
msg = Msg(name=analyst,content="""程序员你好，你现在有空么，我是数据分析师，我有1个数据分析的需求，需要你的协助，数据格式如下："""+ df.to_string())
x = msg
print(x)
for i in range(4):
    x = coder(x)
    x_code = x
    x = analyst(x)

    if "已完成需求沟通"  in x['content'] \
    and "只有当" not in x['content'] \
    and "当我" not in x['content'] \
    and "如果" not in x['content'] \
    and "才会" not in x['content'] :
        print("代码交付成功")
        
        try:
            response_code_output = run_py_code(x_code,show_code=False)
            print('----------- 运行成功 ---------------')
            print(response_code_output)
            print('----------- 分析结论 ---------------')
            msg = Msg(name=coder,content= response_code_output + """以上是代码的运行结果，你作为1个经验分析、擅长洞察数据异动的数据分析师，
            你需要输出1份简洁清晰的数据分析报告（包含5个核心的数据洞察结论）""")
            x = analyst(msg)
                
        except Exception as e:
            print(f"报错代码: {e}")
            print('----------- 代码报错，重新生成 ---------------')
            msg = Msg(name=analyst,content= response_code_output + f"报错代码: {e}" + """#以上代码有误，请重新检查后重新生成，请重新提供完整的代码，不要省略任何必要代码""")
            print(msg)
            x_code = coder(msg)
            response_code_output = run_py_code(x_code,show_code=False)
            print('----------- 重新运行成功 ---------------')
            print(response_code_output)
            msg = Msg(name=coder,content= response_code_output +  """以上是python代码的运行结果，你现在需要输出1份简洁清晰的数据分析报告（包含5个核心的数据洞察结论）""")
            x = analyst(msg)
        break
        
rusult = x['content']

2024-05-20 15:27:10.181 | WARNING  | agentscope.message:__init__:125 - A new field `role` is newly added to the message. Please specify the role of the message. Currently we use a default "assistant" value.
{'id': 'cbfb2ed4e0314f3ab1b0fc0b636d506c', 'timestamp': '2024-05-20 15:27:10', 'name': <agentscope.agents.dialog_agent.DialogAgent object at 0x7e6e5513ebc0>, 'content': '程序员你好，你现在有空么，我是数据分析师，我有1个数据分析的需求，需要你的协助，数据格式如下：       date  Sales_volume_Shanghai  Sales_volume_Beijing  Stores_number_Shanghai  Stores_number_Beijing\n0  20240501                    100                   200                       5                     10\n1  20240502                    100                   300                       6                     11\n2  20240503                    200                   200                       7                     12\n3  20240504                    400                   200                       2                     10', 'role': 'assistant', 'url': None}
程序员: Hello there

## zhipu

In [ ]:
model_config = {
        "model_type": "zhipuai_chat",
        "config_name": "zhipu",
        "model_name": model_chatglm1 ,
        "api_key": api_zz_chatglm,
    "api_url": url_chatglm ,        # 目标url
        "generate_args": {"temperature": 0.5 }
    }


# 读取模型配置
agentscope.init(model_configs=model_config)

# 创建一个对话智能体和一个用户智能体
dialogAgent01 = DialogAgent(name="assistant", 
                          model_config_name="zhipu",  #   model_config_name =  model_config["config_name"]
                          sys_prompt="You are a helpful ai assistant")
userAgent = UserAgent()


# 简单实用 可以当作一个api
from agentscope.message import Msg
msg = Msg(name="小助手",content="你好")
#dialogAgent01(msg)
############################### 目前报错

In [41]:
# 留存：循环范式书写对话链
msg = Msg(name=analyst,content="""程序员你好，你现在有空么，我是数据分析师，有个1个数据分析的需求，需要你的协助""")
# 1v1对话 
print(msg)

# 第一步 需求沟通
print('x0')
x0 = coder(msg)
print('x1')
x1 = analyst(x0)
print('x2')
x2 = coder(x1)
print('x3')
x3 = analyst(x2)

if "已完成需求沟通" not in x3['content']:   
    print('x4')
    x4 = coder(x3)
    print('x5')
    x5 = analyst(x4)
    
    if "已完成需求沟通" not in x5['content']:
        print('x6')
        x6 = coder(x5)
        print('x7')
        x7 = analyst(x6)
        
        if "已完成需求沟通" not in x7['content']:
            print('代码交付失败')
        x2=x6
        
    x2 = x4

code = x2['content']
print('代码交付成功')

2024-05-20 08:17:55.576 | WARNING  | agentscope.message:__init__:125 - A new field `role` is newly added to the message. Please specify the role of the message. Currently we use a default "assistant" value.
{'id': 'a9815ba7e3ff47b5a1e2776e6e9576c5', 'timestamp': '2024-05-20 08:17:55', 'name': <agentscope.agents.dialog_agent.DialogAgent object at 0x7b6c707ed120>, 'content': '程序员你好，你现在有空么，我是数据分析师，有个1个数据分析的需求，需要你的协助', 'role': 'assistant', 'url': None}
x0
程序员: ```python
# 确认分析师需求开始
print("您好，数据分析师。我有空，您可以告诉我具体的数据分析需求是什么？")

# 假设分析师需要计算某个列的平均值，并找出异常值
# 需求：计算df['aa']列的平均值，以及找出该列中大于平均值两倍标准差的数据
# 假设df是一个已经存在的DataFrame，'aa'列存在
mean_value = df['aa'].mean()  # 计算平均值
std_dev = df['aa'].std()  # 计算标准差

# 找出异常值
outliers = df[df['aa'] > mean_value * 2 + std_dev]  # 大于平均值两倍标准差的行

# 输出结果
print(f"列'aa'的平均值是: {mean_value}")
print(f"列'aa'的异常值是: \n{outliers}")

# 请提供df数据，以便我可以直接运行这段代码
```
请提供df数据，或者告诉我如何生成一个符合需求的数据框。
x1
数据分析师: 你好，程序员。首先，感谢你的帮助。为了进行这个分析，我们需要一个包含'aa'列的数据帧df。请根据以下数据格式创建一个DataFrame，它将用于计算和异常值检